In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\marcin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [32]:
from src.yahoo_market_event_scraper import YahooEarningsCalendar
from datetime import datetime
NY = 'America/New_York'
yec = YahooEarningsCalendar()
now = pd.Timestamp.now(tz=NY)

now = datetime.now()
timestamp_now = datetime.timestamp(now)
timestamp_now = round(int(timestamp_now),-2)
print(timestamp_now)
print(datetime.fromtimestamp(timestamp_now))
timestamp_earnings = yec.get_next_earnings_date('XXII')
print(timestamp_earnings)
print(timestamp_earnings/timestamp_now)
print(timestamp_now/timestamp_earnings)
dt_object = datetime.fromtimestamp(timestamp_earnings)
print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

1574630800
2019-11-24 15:26:40
1573171200
0.9990730525530175
1.0009278074757535
dt_object = 2019-11-07 18:00:00
type(dt_object) = <class 'datetime.datetime'>


In [33]:
if now.dayofweek == 4:
    todate = now + datetime.timedelta(days=3)
else:
    todate = now + datetime.timedelta(days=1)
date_from = datetime.datetime(
    now.year, now.month, now.day, 0, 0)
date_to = datetime.datetime(
    todate.year, todate.month, todate.day, 23, 59)
timestamp_earnings = yec.earnings_between(from_date, to_date)

AttributeError: 'datetime.datetime' object has no attribute 'dayofweek'